In [2]:
from DDCASPT2 import DDCASPT2
import pickle, os, shutil
from glob import glob
import numpy as np
from joblib import Parallel, delayed
import pandas as pd
from math import sin, cos, pi
from tqdm.notebook import tqdm
# Geom manipulate
from AaronTools.geometry import Geometry
from tqdm import tqdm
from time import perf_counter

Take [Fe$^{IV}$(O)(H$_2$O)$_4$]$^{+2}$ from Phys. Chem. Chem. Phys., 2018,20, 28786-28795

In [5]:
radius_range=np.linspace(3,5,100)



train_ind,test_ind=radius_range[0::2],radius_range[1::2]
# train_test_split(radius_range, test_size=0.3, random_state=0)
print(len(train_ind),len(test_ind))
with open('train_ind.pickle', 'wb') as handle:
    pickle.dump(train_ind, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('test_ind.pickle', 'wb') as handle:
    pickle.dump(test_ind, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('test_ind.pickle', 'rb') as handle:
    test_ind = pickle.load(handle)

with open('train_ind.pickle', 'rb') as handle:
    train_ind = pickle.load(handle)
    
print(len(train_ind),len(test_ind))    

50 50
50 50


In [6]:
radius_range

array([3.        , 3.02020202, 3.04040404, 3.06060606, 3.08080808,
       3.1010101 , 3.12121212, 3.14141414, 3.16161616, 3.18181818,
       3.2020202 , 3.22222222, 3.24242424, 3.26262626, 3.28282828,
       3.3030303 , 3.32323232, 3.34343434, 3.36363636, 3.38383838,
       3.4040404 , 3.42424242, 3.44444444, 3.46464646, 3.48484848,
       3.50505051, 3.52525253, 3.54545455, 3.56565657, 3.58585859,
       3.60606061, 3.62626263, 3.64646465, 3.66666667, 3.68686869,
       3.70707071, 3.72727273, 3.74747475, 3.76767677, 3.78787879,
       3.80808081, 3.82828283, 3.84848485, 3.86868687, 3.88888889,
       3.90909091, 3.92929293, 3.94949495, 3.96969697, 3.98989899,
       4.01010101, 4.03030303, 4.05050505, 4.07070707, 4.09090909,
       4.11111111, 4.13131313, 4.15151515, 4.17171717, 4.19191919,
       4.21212121, 4.23232323, 4.25252525, 4.27272727, 4.29292929,
       4.31313131, 4.33333333, 4.35353535, 4.37373737, 4.39393939,
       4.41414141, 4.43434343, 4.45454545, 4.47474747, 4.49494

In [7]:
def gen_run(r,path):
    with open(os.path.join(path,f'run.sh'),'w') as g:
        g.write(f"""#!/bin/bash
#This file is a submission script to request the ISAAC resources from Slurm
#SBATCH --account=ACF-UTK0022             # The project account to be charged
#SBATCH --job-name=ironoxo_{r}		       #The name of the job
#SBATCH --nodes=1                     # Number of nodes
#SBATCH --ntasks-per-node=16          # cpus per node
#SBATCH --partition=condo-kvogiatz            # If not specified then default is "campus"
#SBATCH --qos=condo-kvogiatz
#SBATCH --time=1-00:00:00             # Wall time (days-hh:mm:ss)
#SBATCH --error=job.e%J	       # The file where run time errors will be dumped
#SBATCH --output=job.o%J	       # The file where the output of the terminal will be dumped



# PATCH!
# MOVE TO WORKING DIR
echo $SLURM_SUBMIT_DIR
cd $SLURM_SUBMIT_DIR
pwd
# DEFINE WorkDir
mkdir $SLURM_SUBMIT_DIR/$SLURM_JOBID
export WorkDir=$SLURM_SUBMIT_DIR/$SLURM_JOBID
export MOLCAS_WORKDIR="/lustre/isaac/scratch/gjones39"
export MOLCAS="/lustre/isaac/proj/UTK0022/GMJ/OpenMolcas/build"
echo $WorkDir



module purge
module load intel-compilers/2021.2.0
module load hdf5/1.10.8-intel
module load cmake/3.23.2-intel

# DEFINE WorkDir
echo "Starting at $(date)"
echo "Running on hosts: $SLURM_NODELIST"
echo "Running on $SLURM_NNODES nodes."
echo "Running $SLURM_NTASKS tasks."
echo "Current working directory is $(pwd)"

# THE COMMAND

conda activate /lustre/isaac/proj/UTK0022/GMJ/modules/anaconda3/envs/ddcaspt2
pymolcas --new --clean {r}.input -oe {r}.output
# CLEAN-UP AND EXIT
rm -r $SLURM_SUBMIT_DIR/$SLURM_JOBID
echo "Program finished with exit code $? at: $(date)"
""")    

In [8]:

topdir = os.getcwd()



In [9]:
t0 = perf_counter()
geom = Geometry('FeO.xyz')
oxo, iron = geom.atoms[0],geom.atoms[1]

for idxr, r in tqdm(enumerate(radius_range)):

# Create radius subdirectory
    radstr = f"{r:.2f}"
    rad_dir = os.path.join(os.getcwd(),radstr)
    if os.path.exists(rad_dir)==False:
        os.mkdir(rad_dir)
    
        
    # Write xyz
    geom.change_distance(iron,oxo,r)
    geom.write(os.path.join(rad_dir,radstr))
    # if idxr==0:
    #     d = DDCASPT2(rad_dir,'ANO-RCC-VDZP',radstr,10,14,None,previous=None,symmetry=1,spin=4,UHF=True,charge=2,clean=False)(run=False)
    # else:            
        
    previous=os.path.join(topdir,f"{radius_range[idxr-1]:.2f}",f"{radius_range[idxr-1]:.2f}.RasOrb")
    d = DDCASPT2(rad_dir,'ANO-RCC-VDZP',radstr,10,14,None,previous=previous,symmetry=1,spin=4,UHF=True,charge=2,clean=False)(run=False)
    
    gen_run(radstr,rad_dir)
print(perf_counter()-t0)

100it [00:00, 5947.76it/s]

Running on None cores
Found a valid MOLCAS installation at /home/grierjones/Test/build
MOLCAS_WORKDIR is set to /tmp
Running on None cores
Found a valid MOLCAS installation at /home/grierjones/Test/build
MOLCAS_WORKDIR is set to /tmp
Running on None cores
Found a valid MOLCAS installation at /home/grierjones/Test/build
MOLCAS_WORKDIR is set to /tmp
Running on None cores
Found a valid MOLCAS installation at /home/grierjones/Test/build
MOLCAS_WORKDIR is set to /tmp
Running on None cores
Found a valid MOLCAS installation at /home/grierjones/Test/build
MOLCAS_WORKDIR is set to /tmp
Running on None cores
Found a valid MOLCAS installation at /home/grierjones/Test/build
MOLCAS_WORKDIR is set to /tmp
Running on None cores
Found a valid MOLCAS installation at /home/grierjones/Test/build
MOLCAS_WORKDIR is set to /tmp
Running on None cores
Found a valid MOLCAS installation at /home/grierjones/Test/build
MOLCAS_WORKDIR is set to /tmp
Running on None cores
Found a valid MOLCAS installation at /home